# Preprocess

In [ ]:
#coding:utf-8
#the merge part is forked from mapodoufu
import time
import pandas as pd
import numpy as np

# 读取数据
part_1 = pd.read_csv('../data/meinian_round1_data_part1_20180408.txt',sep='$')
part_2 = pd.read_csv('../data/meinian_round1_data_part2_20180408.txt',sep='$')
part_1_2 = pd.concat([part_1,part_2],axis = 0)
part_1_2 = pd.DataFrame(part_1_2).sort_values('vid').reset_index(drop=True)
begin_time = time.time()
print('begin')
# 重复数据的拼接操作
def merge_table(df):
    df['field_results'] = df['field_results'].astype(str)
    if df.shape[0] > 1:
        merge_df = " ".join(list(df['field_results']))
    else:
        merge_df = df['field_results'].values[0]
    return merge_df
# 数据简单处理
print('find_is_copy')
print(part_1_2.shape)
is_happen = part_1_2.groupby(['vid','table_id']).size().reset_index()
# 重塑index用来去重
is_happen['new_index'] = is_happen['vid'] + '_' + is_happen['table_id']
is_happen_new = is_happen[is_happen[0]>1]['new_index']

part_1_2['new_index'] = part_1_2['vid'] + '_' + part_1_2['table_id']

unique_part = part_1_2[part_1_2['new_index'].isin(list(is_happen_new))]
unique_part = unique_part.sort_values(['vid','table_id'])
no_unique_part = part_1_2[~part_1_2['new_index'].isin(list(is_happen_new))]
print('begin')
part_1_2_not_unique = unique_part.groupby(['vid','table_id']).apply(merge_table).reset_index()
part_1_2_not_unique.rename(columns={0:'field_results'},inplace=True)
print('xxx')
tmp = pd.concat([part_1_2_not_unique,no_unique_part[['vid','table_id','field_results']]])
# 行列转换
print('finish')
tmp = tmp.pivot(index='vid',values='field_results',columns='table_id')
#tmp.to_csv('./input/tmp.csv')
print(tmp.shape)
print('totle time',time.time() - begin_time)
tmp.replace('未查',np.nan,inplace=True)
tmp.replace('弃查',np.nan,inplace=True)
import gc
del part_1,part_2,part_1_2,is_happen,is_happen_new,unique_part,no_unique_part,part_1_2_not_unique
gc.collect()
#import pickle
#with open('.\\input\\tmp.pk', 'wb') as f:
#     pickle.dump(tmp, f)

In [ ]:
data_feat = dict()

data_feat['all_feat'] = set(tmp.columns)

In [ ]:
feat_tmp_ditch = []
for s in data_feat['all_feat']:
    if tmp[s].isnull().mean() > 0.95:
        feat_tmp_ditch.append(s)

data_feat['ditch'] = set(feat_tmp_ditch)

In [ ]:
feat_tmp_words = []
for s in data_feat['all_feat'] - data_feat['ditch']:
    if tmp[s].dropna().str.len().mean() > 5:
        feat_tmp_words.append(s)

feat_tmp_words = set(feat_tmp_words) - set(['3301', '100014', '1474', '269011', '191', '1104', '669009', '809045', '669007', '1345'])

data_feat['words'] = feat_tmp_words

In [ ]:
feat_tmp_num_1 = []
for s in data_feat['all_feat'] - data_feat['ditch'] - data_feat['words']:
    try:
        tmp[s].astype(float)
    except:pass
    else:
        feat_tmp_num_1.append(s)

data_feat['num'] = set(feat_tmp_num_1)

feat_tmp_num_2 = [] # suspicious numerical features
for s in data_feat['all_feat'] - data_feat['ditch'] - data_feat['words'] - data_feat['num']:
    if tmp[s].dropna().str.contains('\d').mean() > 0.8:
        feat_tmp_num_2.append(s)

data_feat['num_ish'] = set(feat_tmp_num_2)

In [ ]:
data_feat['cat'] = data_feat['all_feat'] - data_feat['ditch'] - data_feat['words'] - data_feat['num'] - data_feat['num_ish']

In [ ]:
out = tmp[list(data_feat['num_ish'])]
out.replace(r'．', '.', regex = True, inplace = True)
out.replace(r'０', '0', regex = True, inplace = True)
out.replace(r'１', '1', regex = True, inplace = True)
out.replace(r'２', '2', regex = True, inplace = True)
out.replace(r'３', '3', regex = True, inplace = True)
out.replace(r'４', '4', regex = True, inplace = True)
out.replace(r'５', '5', regex = True, inplace = True)
out.replace(r'６', '6', regex = True, inplace = True)
out.replace(r'７', '7', regex = True, inplace = True)
out.replace(r'８', '8', regex = True, inplace = True)
out.replace(r'９', '9', regex = True, inplace = True)
out = out.replace(['/', r'^详见', r'^未做', r'^未查', r'^-+$', r'退检$'], np.nan, regex=True)
out.replace(r'^<(.)*$', r'\1', regex = True, inplace = True)
out.replace(r'^>(.)*$', r'\1', regex = True, inplace = True)
out.replace(r'^阴性$', 0, regex = True, inplace = True)
out.replace(r'^未见$', 0, regex = True, inplace = True)
rep_fun = lambda x: str(np.nanmean([float(i) for i in x.group(0).split()]))

for s in out.columns:
    out.loc[:, s] = out.loc[:, s].str.replace(r'^(([+-]?[0-9]*[.]?[0-9]+)|nan) (([+-]?[0-9]*[.]?[0-9]+)|nan)$', rep_fun)
    out.loc[:, s] = out.loc[:, s].str.replace(r'^(([+-]?[0-9]*[.]?[0-9]+)|nan) (([+-]?[0-9]*[.]?[0-9]+)|nan) (([+-]?[0-9]*[.]?[0-9]+)|nan)$', rep_fun)
    out.loc[:, s] = out.loc[:, s].str.replace(r'^(([+-]?[0-9]*[.]?[0-9]+)|nan) (([+-]?[0-9]*[.]?[0-9]+)|nan) (([+-]?[0-9]*[.]?[0-9]+)|nan) (([+-]?[0-9]*[.]?[0-9]+)|nan)$', rep_fun)

out.replace([r'^<=([+-]?[0-9]*[.]?[0-9]+)$', r'^>=([+-]?[0-9]*[.]?[0-9]+)$',\
            r'^＜([+-]?[0-9]*[.]?[0-9]+)$', r'^＞([+-]?[0-9]*[.]?[0-9]+)$', \
            r'^＜＝([+-]?[0-9]*[.]?[0-9]+)$', r'^＞＝([+-]?[0-9]*[.]?[0-9]+)$'], r'\1', regex = True, inplace=True)

out.replace(r'。', '.', regex = True, inplace = True)

out.replace(r'^\s*(\S*)\s*$', r'\1', regex = True, inplace = True)

out.replace([r'^(.+)S$', r'^(.+)\.$'], r'\1', regex = True, inplace = True)

out.replace(r'^(\d+)\.\.(\d+)$', r'\1.\2', regex = True, inplace = True)

out.replace(['+', '.'], np.nan, regex=False, inplace = True)

out.replace(r'^([+-]?[0-9]*[.]?[0-9]+)-$', r'\1', regex=True, inplace = True)

out.replace(r'^([+-]?[0-9]*[.]?[0-9]+)kpa$', r'\1', regex=True, inplace = True)

# 2404
out['2404'] = out['2404'].str.replace(r'^(\d+) (\S+)$', r'\1')

# 1325
out['1325'][out['1325'].dropna()[out['1325'].dropna().str.match(r'义眼|光感|失明|弃查|指数|正常')].index] = np.nan

out['1325']['0a67044b74d49b6d9e1a9a7c06a0731e'] = 0.8
out['1325']['115e39811df6cdc8c70bd0d58f555558'] = 1.0
out['1325']['1170d101f8f0dd7ab8c005b9fe0bdb68'] = 1.2
out['1325']['76f813be607bf630e29045b07c6e1e1a'] = 1.5
out['1325']['8096e0132a874a4181a7643fd5e3493a'] = np.nan
out['1325']['a8b397c1c014e2db76ec8f360d680283'] = np.nan
out['1325']['af5109d1a04fcdd4ff06bd42745bdfcc'] = np.nan
out['1325']['cca0f6c8bfbd8d0ddf167083a3a29ecd'] = 1.2

# 1815

out['1815']['62bdd9d1540d90a89281ca820f4d69bb'] = np.nan

# 2409

out['2409'] = out['2409'].str.replace(r'^(.+)\(.+\)$', r'\1')

out['2409'] = out['2409'].str.replace(r'^(.+)%$', r'\1')

out['2409'] = out['2409'].str.replace(r'^(.+)% $', r'\1')

# 300009


out['300009']['8a5390a401a3f13d03480f8340140148'] = 45.21

# 1112


out['1112'] = out['1112'].str.replace(r'^(.+) %$', r'\1')

# 269013


rep_fun_2 = lambda x: str(np.nanmean([float(i) for i in x.group(0).split('-')]))
out['269013'] = out['269013'].str.replace(r'^(\d)-(\d)$', rep_fun_2)

# 300078

out['300078']['662d00fbf6636f3b3c311189e99aac30'] = np.nan

# 10004


out['10004']['561416800096bedd9cf6fbad7cef0ef7'] = 3.7

# A701
out['A701']['d0b81951043aa245b46e3f110af60a7f'] = 6.3

# 0424

out['0424'] = out['0424'].str.replace(r'^\D+(\d+)$', r'\1')
out['0424'] = out['0424'].str.replace(r'^(\d+)\D+$', r'\1')

out['0424'] = out['0424'].str.replace('心动过缓', '60')
out['0424'] = out['0424'].str.replace('心率正常', '80')
out['0424'] = out['0424'].str.replace('未见异常', '80')
out['0424'] = out['0424'].str.replace('正常', '80')
out['0424'] = out['0424'].str.replace('窦性心动过缓', '60')
out['0424'] = out['0424'].str.replace('窦性心动过速', '100')

out['0424'] = out['0424'].str.replace('窦性60', '60') # 竟然不是整句匹配

out['0424']['8c6551f10e8610fcadd93970fd9ff482'] = 75

# 312

out['312'] = out['312'].str.replace(r'(?:\s|^)(\d+)-(\d+)(?:\s|$)', lambda x: ' '+rep_fun_2(x)+' ')

out['312'] = out['312'].str.replace(r'^(.+) $', r'\1')
out['312'] = out['312'].str.replace(r'^ (.+)$', r'\1')

out['312'] = out['312'].str.replace('未见', 'nan')

out['312'] = out['312'].str.replace('+', '')
out['312'] = out['312'].str.replace(r'^(.+) $', r'\1')
out['312'] = out['312'].str.replace(r'^ (.+)$', r'\1')

out['312'] = out['312'].str.replace(r'^(([+-]?[0-9]*[.]?[0-9]+)|nan) (([+-]?[0-9]*[.]?[0-9]+)|nan)$', rep_fun)

# 191

out['191'] = out['191'].str.replace('降脂后复查', 'nan')

not_num = lambda this_feat:out[this_feat].dropna()[~out[this_feat].dropna().str.match('^(?:[+-]?([0-9]*[.])?[0-9]+|nan)$')]

# 1320

out['1320'] = out['1320'].str.replace(r' 正常.*$', '')

out['1320'] = out['1320'].str.replace(r'mmHg$', '')

out['1320'] = out['1320'].str.replace(r'^(\d+)\D+$', r'\1')

out['1320'] = out['1320'].str.replace(r'^.*正常.*$', '15')

out['1320'] = out['1320'].str.replace(r'^.*高.*$', '20')

out['1320'][not_num('1320').index] = 'nan'

# 313

out['313']['efc9d79c3cfca3c5eaef9ebce428bac2'] = 189

# 300017
out['300017']['f32fc7e0d65c3f0ef11dba36284b99f4'] = 1.496

# 2403

out['2403'] = out['2403'].str.replace(r'^(\d+)\s\S+$', r'\1')

# 2372

out['2372']['e50b5c308f0cb33351479f92667f933b'] = (2.79 + 2.20)/2

# 300013

out['300013']['53027913d6c15b82098658f3b358ffad'] = (6.31 + 0.45) / 2

# 300076
out['300076']['662d00fbf6636f3b3c311189e99aac30'] = np.nan

# 192

out['192']['f9d1b02f5579d4e73ac833775724fbc2'] = np.nan

# 1319

out['1319'] = out['1319'].str.replace(r' 正常范围.*$', '')

out['1319'] = out['1319'].str.replace(r'mmHg$', '')

out['1319'] = out['1319'].str.replace(r'^(\d+)\D+$', r'\1')

out['1319'] = out['1319'].str.replace(r'^.*正常.*$', '15')

out['1319'] = out['1319'].str.replace(r'^.*高.*$', '20')

out['1319'][not_num('1319').index] = 'nan'

## 1322
out['1322'][out['1322'].dropna()[out['1322'].dropna().str.match(r'义眼|光感|失明|弃查|指数|正常|无光感|手动|因无法配合不能检测|未要求检查')].index] = np.nan

out['1322'][not_num('1322').index] = ['0.9', 'nan', '0.8', 'nan', '1.2', '1.2', 'nan', '0.8', '1.0', 'nan', '0.9',\
                                     'nan', 'nan']

## 1326

out['1326'][out['1326'].dropna()[out['1326'].dropna().str.match(r'义眼|光感|失明|弃查|指数|正常|无光感|手动|因无法配合不能检测|未要求检查')].index] = np.nan

out['1326'][not_num('1326').index] = ['1.2', '1.0', '1.5', '1.38', 'nan', 'nan', '0.85', 'nan', '1.2', 'nan', '1.0']

## 1321

out['1321'][out['1321'].dropna()[out['1321'].dropna().str.match(r'义眼|光感|失明|弃查|指数|正常|无光感|手动|因无法配合不能检测|未要求检查')].index] = np.nan

out['1321'][not_num('1321').index] = ['0.6', 'nan', '0.8', 'nan', '1.5', '1.2', 'nan', \
                                    '0.6', '1.0', 'nan', '1.0', '0.55', 'nan']

In [ ]:
num_ish = out.astype(float)
num_ish = (num_ish - num_ish.mean(axis = 0))/ num_ish.std(axis = 0)
num_ish.columns = [x + '*' for x in num_ish.columns]

# Run

In [ ]:
l_word = tmp[list(data_feat['words'])]
l_cat = tmp[list(data_feat['cat'])]

In [ ]:
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import jieba
jieba.load_userdict("../code/user_dict.py")
MAX_FEATURE=50

# stop words are the most common words in a language
# which should be filtered out
def stopwordslist(filepath):
    stopword = [line.strip() for line in open(filepath, 'rb').readlines()]
    return stopword
stopwords = stopwordslist('../code/stop_words.py')

# word segmentation using jieba
def seg_sentence(sentence):
    if (sentence is None) or (sentence != sentence): return ""
    try:
        sentence_seged = jieba.cut(sentence.strip())
        # sentence_seged = [w.encode('utf8') for w in sentence_seged]
        sentence_seged = [w for w in sentence_seged if w not in stopwords]
        sentence_seged = " ".join(sentence_seged)
        return sentence_seged
    except:
        print(sentence)
        return ""

def word2tfidf(sentlist,min_df,max_df):
    vectorizer = CountVectorizer(token_pattern='(?u)\\b\\w+\\b',min_df = min_df, max_df =max_df, max_features = MAX_FEATURE)
    transformer = TfidfTransformer()
    count = vectorizer.fit_transform(sentlist)
    tf_idf = transformer.fit_transform(count)
    # print(len(vectorizer.get_feature_names()))
    # print(vectorizer.get_feature_names())
    return tf_idf.toarray().tolist()



# 处理word类型的数据
print("[PROCESS] Word file loaded, shape = " + str(l_word.shape))
dimension = 0
for i in range(l_word.shape[1]):
    if i%10==0: print("Word file " + str(i) +", dimention = " + str(dimension))
    word_list = l_word.iloc[:,i].values
    word_list = [seg_sentence(sent) for sent in word_list]
    tfidf = word2tfidf(word_list,0.00002,0.50)
    dimension += len(tfidf[0])
    l_word.iloc[:, i] = tfidf

In [ ]:
from sklearn.cluster import KMeans
word_2_label = []
for col in range(l_word.shape[1]):
    print(col)
    vec_list = []
    for i in range(l_word.shape[0]):
        vec_list.append(l_word.iloc[i,col])
    vec_array = np.array(vec_list)
    kmeans = KMeans(n_clusters=10, random_state=0)
    kmeans.fit(vec_array)
    word_2_label.append(pd.DataFrame(kmeans.predict(vec_array)))
word_label = pd.concat(word_2_label,axis=1)
word_label.columns = l_word.columns
word_label.index = l_word.index
del word_2_label,vec_list,vec_array,l_word
gc.collect()

In [ ]:
l_cat = tmp[list(data_feat['cat'])]
print("[PROCESS] Word file transformed, dimention = " + str(dimension))
# 处理cat类型的数据
print("[PROCESS] Cat file loaded, shape = " + str(l_cat.shape))
dimension = 0
for i in range(l_cat.shape[1]):
    if i%10==0: print("Word file " + str(i) +", dimention = " + str(dimension))
    word_list = l_cat.iloc[:,i].values
    word_list = [seg_sentence(sent) for sent in word_list]
    try:
        tfidf = word2tfidf(word_list,0.0,1.00)
    except:
        print(i)
    dimension += len(tfidf[0])
    l_cat.iloc[:, i] = tfidf

print("[PROCESS] Cat file transformed, dimention = " + str(dimension))

cat_2_label = []
for col in range(l_cat.shape[1]):
    print(col)
    vec_list = []
    for i in range(l_cat.shape[0]):
        vec_list.append(l_cat.iloc[i,col])
    vec_array = np.array(vec_list)
    kmeans = KMeans(n_clusters=5, random_state=0)
    kmeans.fit(vec_array)
    cat_2_label.append(pd.DataFrame(kmeans.predict(vec_array)))

In [ ]:
cat_label = pd.concat(cat_2_label,axis=1)
cat_label.columns = l_cat.columns
cat_label.index = l_cat.index

del cat_2_label,vec_list,vec_array,l_cat
gc.collect()

In [ ]:
num_ish = num_ish.fillna(num_ish.mean())
num_list = []
for i in data_feat['num']:
    num_list.append(i)
num_X = tmp[num_list]
num_X_list = []
for i in range(num_X.shape[1]):
    num_X_list.append(num_X.iloc[:,i].astype(float, inplace=True))
num_X = pd.concat(num_X_list,axis = 1)
num_X.fillna(num_X.mean(),inplace = True)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
std_num = pd.DataFrame(scaler.fit_transform(num_X))
std_num.index = num_X.index
std_num.columns = num_X.columns
std_num_X = pd.concat([std_num,num_ish],axis=1)

In [ ]:
all_cat = pd.concat([word_label,cat_label],axis=1)
all_X = pd.concat([all_cat,std_num_X],axis=1)

In [ ]:
dtype = {'vid':str,'收缩压':object,'舒张压':object,'血清甘油三酯':object,'血清高密度脂蛋白':object,'血清低密度脂蛋白':object}
train_data = pd.read_csv('..\\data\\meinian_round1_train_20180408.csv',encoding='gb2312',dtype=dtype)
train_data = train_data.set_index('vid')
train_data.replace('未查',np.nan,inplace=True)
train_data.replace('弃查',np.nan,inplace=True)
train_data.iloc[:,2] = train_data.iloc[:,2].str.strip('轻度乳糜').str.strip('+').str.strip('>').str.strip('=').str.strip('-')
train_data.iloc[:,3] = train_data.iloc[:,3].str.strip('+').str.strip('>').str.strip('=').str.strip('-')
train_data.iloc[:,4] = train_data.iloc[:,4].str.strip('+').str.strip('>').str.strip('=').str.strip('-')
train_data.replace('2.2.8','2.28',inplace=True)
train_data.replace('<=5.0',5.0,inplace=True)
train_data.replace('＞＝１.０３０',1.030,inplace=True)
train_data.replace('>=1.030',1.030,inplace=True)
train_data.replace('未做',np.nan,inplace=True)
train_data.replace('阴性',np.nan,inplace=True)
train_data.replace('1.015.',1.015,inplace=True)
train_data.replace('8.53.',8.53,inplace=True)
train_data.replace('2.1.',2.1,inplace=True)
train_data.replace('---',np.nan,inplace=True)
train_data.replace('16.2-',16.1999,inplace=True)
train_data.replace('189 脂血',189,inplace=True)
train_data.replace('-',np.nan,inplace=True)
train_data.replace('降脂后复查',np.nan,inplace=True)
train_data.replace('无',np.nan,inplace=True)
train_data.replace('3。89',3.89,inplace=True)
train_data.replace('16.7.07',16.7,inplace=True)
train_data.replace('',np.nan,inplace=True)
train_data.replace('77..21',77.21,inplace=True)
train_data.replace('0-5 3',3,inplace=True)
train_data.replace('6-1',6.1,inplace=True)
train_data.replace('未见',np.nan,inplace=True)
train_data.replace('5..0',5.0,inplace=True)
train_data.replace('详见报告单',np.nan,inplace=True)
train_data.replace('Ⅱ',2,inplace=True)
train_data.replace('Ⅲ',3,inplace=True)
train_data.replace('Ⅰ',1,inplace=True)
train_data.replace('Ⅲ度',3,inplace=True)
train_data.replace('中度',3,inplace=True)
train_data.replace('Ⅳ',4,inplace=True)
train_data.replace('Ⅱ度',2,inplace=True)
train_data.replace('未见异常',np.nan,inplace=True)
train_data.replace('III',3,inplace=True)
train_data.replace('正常',np.nan,inplace=True)
train_data.replace('见TCT',np.nan,inplace=True)
train_data.replace('iii°',3,inplace=True)
train_data.replace('微混',1,inplace=True)
train_data.replace('ii°',2,inplace=True)
train_data.replace('II',2,inplace=True)
train_data.replace('II',2,inplace=True)
train_data = train_data.astype(float,inplace=True)
train_data.replace(0,np.nan,inplace=True)
train_data.dropna(inplace=True)
train_data = np.log10(train_data)

In [ ]:
train = train_data.join(all_X)
test_vid = pd.read_csv('..\\data\\meinian_round1_test_b_20180505.csv',encoding = 'gb2312')
test_vid.set_index('vid',inplace = True)
X_test = test_vid.join(all_X).iloc[:,5:]

In [ ]:
del train_data,test_vid,num_X,std_num_X,all_cat,all_X,cat_label,word_label,tmp
gc.collect()

In [ ]:
train.shape

In [ ]:
all_cat.shape

# Model

In [ ]:
import lightgbm as lgb
params = {
    'objective': 'regression_l2',
    'boosting_type': 'gbdt',
    'metric' : 'mse',
    'num_leaves': 80,
    'learning_rate': 0.03,
    'n_estimators':1000,
    'feature_fraction': 0.3,
    'bagging_freq': 2,
    'num_threads': 4,
    'reg_alpha':0.9,
    'reg_lambda':0.0005,
    'max_bin':100
}
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from random import randrange

In [ ]:
result_list = []
mse_list = []
model_list = []
#bagging_round = 100
bagging_round = 1
for rnd in range(bagging_round):
    print('%d round'%(rnd+1))
    seed = randrange(1,10000,1)
    print('seed = ',seed)
    X_train, X_cv, y_train, y_cv = train_test_split(train.iloc[:,5:], train.iloc[:,0:5], test_size=0.25, random_state=seed)
    MAX_ROUNDS = 100000
    val_pred = []
    test_pred = []
    cat_var = range(0,163)
    for i in range(5):
        print('%d round'%(rnd+1))
        print("=" * 50)
        print("Step %d" % (i+1))
        print("=" * 50)
        #dtrain = lgb.Dataset(X_train, label = y_train.iloc[:,i])
        dtrain = lgb.Dataset(X_train, label = y_train.iloc[:,i],categorical_feature=cat_var)
        dval = lgb.Dataset(X_cv, label = y_cv.iloc[:,i],reference=dtrain,categorical_feature=cat_var)
        bst = lgb.train(
            params, dtrain, num_boost_round=MAX_ROUNDS,
            valid_sets=[dtrain, dval],categorical_feature=cat_var,early_stopping_rounds=50)
        val_pred.append(bst.predict(X_cv, num_iteration=bst.best_iteration or MAX_ROUNDS))
        test_pred.append(bst.predict(X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))
        model_list.append(bst)
    mse = 0
    for i in range(len(val_pred)):
        mse = mse + mean_squared_error(y_cv.iloc[:,i],val_pred[i].reshape(-1,1))/5
    print('cv mse:',mse)
    mse_list.append(mse)
    test_exp = []
    for i in range(len(test_pred)):
        test_exp.append(pd.DataFrame(10**test_pred[i]))
    test_exp = pd.concat(test_exp,axis=1)
    test_exp.columns = ['收缩压', '舒张压', '血清甘油三酯', '血清高密度脂蛋白', '血清低密度脂蛋白']
    test_exp.index = X_test.index
    result_list.append(test_exp)
a = result_list[0]/bagging_round
for n in range(bagging_round-1):
    a = a+result_list[n+1]/bagging_round
import datetime
a.to_csv('..\\submit\\submit_'+datetime.datetime.now().strftime('%Y%m%d_%H%M%S')+'.csv',header=False)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
y_train.columns

In [ ]:
name_list = ['Systolic Pressure','Diastolic Pressure','Triglyceride','S-HDL','S-LDL']

In [ ]:
for i in range(y_train.shape[1]):
    lgb.plot_importance(model_list[i],max_num_features=20)
    plt.title(name_list[i])

In [ ]:
import pandas as pd
d = pd.read_csv('E:\\Data Science Laboratory\\kaggle\\train_cleaned_v1.csv')

In [ ]:
d.shape